In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

/home/renmeng/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
VAL_SPLIT_DATE = '2016-09-15'   # Cutoff date for validation split
USE_SEASONAL_FEATURES = True
LEARING_RATE = 0.007
CV_ONLY = False
ADD_OTHER = True

In [3]:
properties16 = pd.read_csv('../data/properties_2016.csv/properties_2016.csv', low_memory = False)

# Number of properties in the zip
zip_count = properties16['regionidzip'].value_counts().to_dict()
# Number of properties in the city
city_count = properties16['regionidcity'].value_counts().to_dict()
# Median year of construction by neighborhood
medyear = properties16.groupby('regionidneighborhood')['yearbuilt'].aggregate('median').to_dict()
# Mean square feet by neighborhood
meanarea = properties16.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()
# Neighborhood latitude and longitude
medlat = properties16.groupby('regionidneighborhood')['latitude'].aggregate('median').to_dict()
medlong = properties16.groupby('regionidneighborhood')['longitude'].aggregate('median').to_dict()

train = pd.read_csv("../data/train_2016_v2.csv/train_2016_v2.csv")
for c in properties16.columns:
    properties16[c]=properties16[c].fillna(-1)
    if properties16[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties16[c].values))
        properties16[c] = lbl.transform(list(properties16[c].values))
train_df = train.merge(properties16, how='left', on='parcelid')
#2016-09-15
select_qtr4 = pd.to_datetime(train_df["transactiondate"]) >= VAL_SPLIT_DATE
if USE_SEASONAL_FEATURES:
    basedate = pd.to_datetime('2015-11-15').toordinal()
    
citystd = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("std").to_dict()
zipstd = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("std").to_dict()
hoodstd = train_df[~select_qtr4].groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()


        
dropvars = ['airconditioningtypeid', 'buildingclasstypeid',
            'buildingqualitytypeid', 'regionidcity']
droptrain = ['parcelid', 'logerror', 'transactiondate']
droptest = ['ParcelId']

In [4]:
train_df = train.merge(properties16, how='left', on='parcelid')

In [5]:
train_df.shape

(90275, 60)

In [425]:
def calculate_features_2(df):
    cols = ['parcelid','logerror', 'transactiondate', 'bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'lotsizesquarefeet',\
             'roomcnt', 'fips', 'latitude', 'longitude', 'propertycountylandusecode', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcounty', \
           'regionidcity', 'regionidzip', 'regionidneighborhood',  'yearbuilt', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', \
           'taxamount', 'assessmentyear']
    df = df[cols]
    df['N-zip_count'] = df['regionidzip'].map(zip_count)
    # Number of properties in the city
    df['N-city_count'] = df['regionidcity'].map(city_count)
    # Does property have a garage, pool or hot tub and AC?
     # Mean square feet of neighborhood properties
    df['mean_area'] = df['regionidneighborhood'].map(meanarea)
    # Median year of construction of neighborhood properties
    df['med_year'] = df['regionidneighborhood'].map(medyear)
    # Neighborhood latitude and longitude
    df['med_lat'] = df['regionidneighborhood'].map(medlat)
    df['med_long'] = df['regionidneighborhood'].map(medlong)
    df['hood_std'] = df['regionidneighborhood'].map(hoodstd)

    df['zip_std'] = df['regionidzip'].map(zipstd)
    df['city_std'] = df['regionidcity'].map(citystd)
    df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
    df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)

In [186]:
ADD_OTHER = False

In [165]:
train_df.columns

Index([u'parcelid', u'logerror', u'transactiondate', u'airconditioningtypeid',
       u'architecturalstyletypeid', u'basementsqft', u'bathroomcnt',
       u'bedroomcnt', u'buildingclasstypeid', u'buildingqualitytypeid',
       u'calculatedbathnbr', u'decktypeid', u'finishedfloor1squarefeet',
       u'calculatedfinishedsquarefeet', u'finishedsquarefeet12',
       u'finishedsquarefeet13', u'finishedsquarefeet15',
       u'finishedsquarefeet50', u'finishedsquarefeet6', u'fips',
       u'fireplacecnt', u'fullbathcnt', u'garagecarcnt', u'garagetotalsqft',
       u'hashottuborspa', u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequart

In [506]:
JOIN = True

In [636]:
USE_SEASONAL_FEATURES = True
JOIN = True
DELETE = True
DUMMY = True
MISSING = False
TEST = True

In [643]:
sample_submission = pd.read_csv('../data/sample_submission.csv', low_memory = False)


droptest += ['transactiondate']

test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )


In [644]:
test_df['transactiondate'] = '2016-10-31'

In [645]:
from scipy.stats import mode
# 中位数
def mode_function(df):
    counts = mode(df)
    return counts[0][0]
def calculate_features(df):
    
    if USE_SEASONAL_FEATURES:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)
    # Number of properties in the zip
#     df['N-zip_count'] = df['regionidzip'].map(zip_count)
#     Number of properties in the city
#     df['N-city_count'] = df['regionidcity'].map(city_count)
#     Does property have a garage, pool or hot tub and AC?
#     df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
#                          (df['pooltypeid10']>0) & \
#                          (df['airconditioningtypeid']!=5))*1 

    # More features
    # Mean square feet of neighborhood properties
#     df['mean_area'] = df['regionidneighborhood'].map(meanarea)
#     Median year of construction of neighborhood properties
#     df['med_year'] = df['regionidneighborhood'].map(medyear)
#     Neighborhood latitude and longitude
#     df['med_lat'] = df['regionidneighborhood'].map(medlat)
#     df['med_long'] = df['regionidneighborhood'].map(medlong)

#     df['zip_std'] = df['regionidzip'].map(zipstd)
#     df['city_std'] = df['regionidcity'].map(citystd)
#     df['hood_std'] = df['regionidneighborhood'].map(hoodstd)
    
    if JOIN:
        tmp_log = df.groupby('regionidzip')['logerror'].agg([('mean_',np.mean),
                                                    ('std_',np.std),('count_', 'count')]).reset_index()
        df = pd.merge(df, tmp_log, on=['regionidzip'], how='left')
        
        tmp = df.groupby('regionidzip')['yearbuilt'].agg([('mean_year',np.mean),
                                                    ('std_year',np.std)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        tmp = df.groupby('regionidzip')['latitude'].agg([('mean_latitude',np.mean),
                                                    ('std_latitude',np.std), ('median_latitude', np.median)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        tmp = df.groupby('regionidzip')['longitude'].agg([('mean_longitude',np.mean),
                                                    ('std_longitude',np.std), ('median_longitude', np.median)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        
        tmp = df.groupby('regionidzip')['rawcensustractandblock'].agg([('mean_census',np.mean),
                                                    ('std_census',np.std)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        tmp = df.groupby('regionidzip')['taxamount'].agg([('mean_tax',np.mean),('max_tax', np.max), ('min_tax', np.min),
                                                    ('std_tax',np.std), ('mode_tax', mode_function), ('median_tax',np.median)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        tmp = df.groupby('regionidzip')['cos_season'].agg([('mean_season',np.mean),
                                                    ('std_season',np.std)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
#         ###
        
        tmp = df.groupby('regionidcity')['logerror'].agg([('mean_city',np.mean),
                                                    ('std_city',np.std),('count_city', 'count')]).reset_index()
        df = pd.merge(df, tmp, on=['regionidcity'], how='left')
        
#         tmp = df.groupby('regionidcity')['yearbuilt'].agg([('mean_city_year',np.mean),
#                                                     ('std_city_year',np.std)]).reset_index()
#         df = pd.merge(df, tmp, on=['regionidcity'], how='left')
        
#         tmp = df.groupby('regionidcity')['latitude'].agg([('mean_city_latitude',np.mean),
#                                                     ('std_city_latitude',np.std), ('median_city_latitude', np.median)]).reset_index()
#         df = pd.merge(df, tmp, on=['regionidcity'], how='left')
#         tmp = df.groupby('regionidcity')['longitude'].agg([('mean_city_longitude',np.mean),
#                                                     ('std_city_longitude',np.std), ('median_city_longitude', np.median)]).reset_index()
#         df = pd.merge(df, tmp, on=['regionidcity'], how='left')
        
        ###
        
        tmp = df.groupby('propertylandusetypeid')['logerror'].agg([('mean_land',np.mean),
                                                    ('std_land',np.std),('count_land', 'count')]).reset_index()
        df = pd.merge(df, tmp, on=['propertylandusetypeid'], how='left')
        del df['propertylandusetypeid']
        
        tmp = df.groupby('regionidzip')['std_land'].agg([('mean_std_land',np.mean),
                                                    ('std_std_land',np.std)]).reset_index()
        df = pd.merge(df, tmp, on=['regionidzip'], how='left')
        
        
        ###
        
        tmp = df.groupby('regionidneighborhood')['logerror'].agg([('mean_nei',np.mean),
                                                    ('std_nei',np.std),('count_nei', 'count')]).reset_index()
        df = pd.merge(df, tmp, on=['regionidneighborhood'], how='left')
        
        ##
        
        tmp = df.groupby('regionidcounty')['logerror'].agg([('mean_county',np.mean),
                                                    ('std_county',np.std),('count_county', 'count')]).reset_index()
        df = pd.merge(df, tmp, on=['regionidcounty'], how='left')
        if DELETE:
            del df['regionidzip']
            del df['regionidneighborhood']
            del df['regionidcounty']
#             del df['regionidcity']
        print df.shape

    
    if ADD_OTHER:
        #life of property
        df['N-life'] = 2018 - df['yearbuilt']
        #Ratio of tax of property over parcel
        df['N-ValueRatio'] = df['taxvaluedollarcnt']/df['taxamount']

        #TotalTaxScore
        df['N-TaxScore'] = df['taxvaluedollarcnt']*df['taxamount']

        #polnomials of tax delinquency year
        df["N-taxdelinquencyyear-2"] = df["taxdelinquencyyear"] ** 2
        df["N-taxdelinquencyyear-3"] = df["taxdelinquencyyear"] ** 3

        #Length of time since unpaid taxes
        df['N-life2'] = 2018 - df['taxdelinquencyyear']

        #error in calculation of the finished living area of home
        df['N-LivingAreaError'] = df['calculatedfinishedsquarefeet']/df['finishedsquarefeet12']

        #proportion of living area
        df['N-LivingAreaProp'] = df['calculatedfinishedsquarefeet']/df['lotsizesquarefeet']
        df['N-LivingAreaProp2'] = df['finishedsquarefeet12']/df['finishedsquarefeet15']

        #Amout of extra space
        df['N-ExtraSpace'] = df['lotsizesquarefeet'] - df['calculatedfinishedsquarefeet'] 
        df['N-ExtraSpace-2'] = df['finishedsquarefeet15'] - df['finishedsquarefeet12'] 

        #Total number of rooms
        df['N-TotalRooms'] = df['bathroomcnt']*df['bedroomcnt']

        #Average room size
        df['N-AvRoomSize'] = df['calculatedfinishedsquarefeet']/df['roomcnt'] 

        # Number of Extra rooms
        df['N-ExtraRooms'] = df['roomcnt'] - df['N-TotalRooms'] 

        #Ratio of the built structure value to land area
        df['N-ValueProp'] = df['structuretaxvaluedollarcnt']/df['landtaxvaluedollarcnt']

        #Does property have a garage, pool or hot tub and AC?

        df["N-location"] = df["latitude"] + df["longitude"]
        df["N-location-2"] = df["latitude"]*df["longitude"]
        df["N-location-2round"] = df["N-location-2"].round(-4)

        df["N-latitude-round"] = df["latitude"].round(-4)
        df["N-longitude-round"] = df["longitude"].round(-4)
        
        #Indicator whether it has AC or not
        df['N-ACInd'] = (df['airconditioningtypeid']!=5)*1

        #Indicator whether it has Heating or not 
        df['N-HeatInd'] = (df['heatingorsystemtypeid']!=13)*1

        #There's 25 different property uses - let's compress them down to 4 categories
#         df['N-PropType'] = df.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 246 : "Mixed", 247 : "Mixed", 248 : "Mixed", 260 : "Home", 261 : "Home", 262 : "Home", 263 : "Home", 264 : "Home", 265 : "Home", 266 : "Home", 267 : "Home", 268 : "Home", 269 : "Not Built", 270 : "Home", 271 : "Home", 273 : "Home", 274 : "Other", 275 : "Home", 276 : "Home", 279 : "Home", 290 : "Not Built", 291 : "Not Built" })
        #polnomials of the variable
        df["N-structuretaxvaluedollarcnt-2"] = df["structuretaxvaluedollarcnt"] ** 2
        df["N-structuretaxvaluedollarcnt-3"] = df["structuretaxvaluedollarcnt"] ** 3

        #Average structuretaxvaluedollarcnt by city
        group = df.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
        df['N-Avg-structuretaxvaluedollarcnt'] = df['regionidcity'].map(group)

        #Deviation away from average
        df['N-Dev-structuretaxvaluedollarcnt'] = abs((df['structuretaxvaluedollarcnt'] - df['N-Avg-structuretaxvaluedollarcnt']))/df['N-Avg-structuretaxvaluedollarcnt']
    
    if MISSING:
        miss = ['propertyzoningdesc', 'unitcnt', 'buildingqualitytypeid', \
                'heatingorsystemtypeid', 'regionidneighborhood', 'garagecarcnt', \
                'garagetotalsqft', 'airconditioningtypeid', 'numberofstories',\
                'poolcnt', 'pooltypeid7', 'fireplacecnt', 'threequarterbathnbr',\
                'finishedfloor1squarefeet', 'finishedsquarefeet50',\
                'finishedsquarefeet15', 'yardbuildingsqft17', 'hashottuborspa',\
                'taxdelinquencyyear', 'taxdelinquencyflag', 'pooltypeid10',\
                'pooltypeid2', 'poolsizesum', 'finishedsquarefeet6', 'decktypeid',\
                'buildingclasstypeid', 'finishedsquarefeet13',\
                'typeconstructiontypeid', 'architecturalstyletypeid',\
                'fireplaceflag', 'yardbuildingsqft26', 'basementsqft', 'storytypeid']
        for c in miss:
            if c in df.columns.values:
                del df[c]
    return df

In [646]:
train_df, df_test = calculate_features(train_df, test_df)
print train_df.shape
print df_test.shape

x_valid = train_df.drop(dropvars+droptrain, axis=1)[select_qtr4]
# x_valid = train_df.drop(droptrain, axis=1)[select_qtr4]
y_valid = train_df["logerror"].values.astype(np.float32)[select_qtr4]

print('Shape valid X: {}'.format(x_valid.shape))
print('Shape valid y: {}'.format(y_valid.shape))

train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
print('\nFull training set after removing outliers, before dropping vars:')     
print('Shape training set: {}\n'.format(train_df.shape))

train_df=train_df[~select_qtr4]
x_train=train_df.drop(dropvars+droptrain, axis=1)
# x_train=train_df.drop(droptrain, axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)
n_train = x_train.shape[0]
print('Training subset after removing outliers:')     
print('Shape train X: {}'.format(x_train.shape))
print('Shape train y: {}'.format(y_train.shape))


(90275, 93)
(90275, 93)
(2985217, 92)
Shape valid X: (14304, 86)
Shape valid y: (14304,)

Full training set after removing outliers, before dropping vars:
Shape training set: (88528, 93)

Training subset after removing outliers:
Shape train X: (74478, 86)
Shape train y: (74478,)


/home/renmeng/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [647]:
x_test = df_test.drop(dropvars+droptest, axis=1)
x_test.shape

(2985217, 86)

In [559]:
#     Does property have a garage, pool or hot tub and AC

In [600]:
# split feature
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': 0.007,
    'max_depth': 7, 
    'subsample': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 2.0,
    'alpha': 0.8,
    'colsample_bytree': 0.3,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)
# if not CV_ONLY:
#     dtest = xgb.DMatrix(x_test)
#     dtest17 = xgb.DMatrix(x_test17)
#     del x_test

In [469]:
# original
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': 0.007,
    'max_depth': 7, 
    'subsample': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 5.0,
    'alpha': 0.65,
    'colsample_bytree': 0.5,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)
# if not CV_ONLY:
#     dtest = xgb.DMatrix(x_test)
#     dtest17 = xgb.DMatrix(x_test17)
#     del x_test

In [122]:
# add_other is True
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': 0.007,
    'max_depth': 8, 
    'subsample': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 5.0,
    'alpha': 0.65,
    'colsample_bytree': 0.3,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)
# if not CV_ONLY:
#     dtest = xgb.DMatrix(x_test)
#     dtest17 = xgb.DMatrix(x_test17)
#     del x_test

In [601]:
num_boost_rounds = 1200
early_stopping_rounds = 100
print('Boosting rounds: {}'.format(num_boost_rounds))
print('Early stoping rounds: {}'.format(early_stopping_rounds))
evals = [(dtrain,'train'),(dvalid,'eval')]
model = xgb.train(xgb_params, dtrain, num_boost_round=int(num_boost_rounds),
                  evals=evals, early_stopping_rounds=early_stopping_rounds, 
                  verbose_eval=100)

Boosting rounds: 1200
Early stoping rounds: 100
[0]	train-mae:0.053447	eval-mae:0.065274
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[100]	train-mae:0.052753	eval-mae:0.064792
[200]	train-mae:0.052301	eval-mae:0.064551
[300]	train-mae:0.051967	eval-mae:0.064409
[400]	train-mae:0.051686	eval-mae:0.064316
[500]	train-mae:0.051449	eval-mae:0.06426
[600]	train-mae:0.051235	eval-mae:0.064236
[700]	train-mae:0.051029	eval-mae:0.064208
[800]	train-mae:0.050846	eval-mae:0.064202
Stopping. Best iteration:
[739]	train-mae:0.050954	eval-mae:0.064201



In [381]:
import operator
fea = sorted(model.get_fscore().items(), key=operator.itemgetter(1), reverse=True)
col = [ele[0] for ele in fea]
col = col[:-10]
fea

[('taxamount', 4490),
 ('yearbuilt', 4344),
 ('calculatedfinishedsquarefeet', 4271),
 ('structuretaxvaluedollarcnt', 4183),
 ('cos_season', 4178),
 ('sin_season', 4144),
 ('lotsizesquarefeet', 4122),
 ('landtaxvaluedollarcnt', 4112),
 ('taxvaluedollarcnt', 3932),
 ('latitude', 3336),
 ('longitude', 3312),
 ('finishedsquarefeet12', 2968),
 ('mean_', 2001),
 ('std_', 1819),
 ('propertyzoningdesc', 1673),
 ('std_latitude', 1513),
 ('rawcensustractandblock', 1466),
 ('mean_std_land', 1416),
 ('mode_tax', 1403),
 ('count_', 1321),
 ('mean_year', 1302),
 ('med_lat', 1298),
 ('censustractandblock', 1271),
 ('mean_city', 1249),
 ('std_year', 1223),
 ('std_std_land', 1222),
 ('std_longitude', 1198),
 ('mean_tax', 1195),
 ('med_long', 1189),
 ('std_census', 1179),
 ('max_tax', 1172),
 ('std_tax', 1162),
 ('min_tax', 1138),
 ('bedroomcnt', 1136),
 ('bathroomcnt', 1089),
 ('median_tax', 1077),
 ('mean_census', 1035),
 ('std_city', 947),
 ('propertycountylandusecode', 856),
 ('count_city', 811),
 (

In [180]:
x_train = x_train[col]
x_valid = x_valid[col]

In [78]:
fea

[('taxamount', 4447),
 ('landtaxvaluedollarcnt', 4258),
 ('latitude', 3983),
 ('lotsizesquarefeet', 3976),
 ('sin_season', 3905),
 ('cos_season', 3840),
 ('yearbuilt', 3822),
 ('structuretaxvaluedollarcnt', 3751),
 ('taxvaluedollarcnt', 3719),
 ('longitude', 3688),
 ('calculatedfinishedsquarefeet', 3563),
 ('finishedsquarefeet12', 2687),
 ('regionidzip', 2595),
 ('zip_std', 2578),
 ('N-zip_count', 2523),
 ('rawcensustractandblock', 2069),
 ('propertyzoningdesc', 1968),
 ('mean_area', 1685),
 ('censustractandblock', 1561),
 ('N-city_count', 1442),
 ('med_year', 1420),
 ('regionidcity', 1419),
 ('city_std', 1380),
 ('hood_std', 1279),
 ('bedroomcnt', 1149),
 ('med_long', 1111),
 ('med_lat', 1095),
 ('regionidneighborhood', 1078),
 ('propertycountylandusecode', 940),
 ('bathroomcnt', 927),
 ('garagetotalsqft', 709),
 ('finishedsquarefeet15', 708),
 ('propertylandusetypeid', 640),
 ('calculatedbathnbr', 518),
 ('buildingqualitytypeid', 475),
 ('poolcnt', 474),
 ('heatingorsystemtypeid', 39

In [648]:

train = pd.read_csv("../data/train_2016_v2.csv/train_2016_v2.csv")
train = train[['parcelid', 'logerror']]

test_df = pd.merge(test_df, train.rename(columns = {'parcelid': 'ParcelId'}), on=['ParcelId'], how='left')

test_df['transactiondate'] = '2016-10-31'
calculate_features(test_df)
x_test = test_df.drop(droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_10 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
test_df['transactiondate'] = '2016-11-30'
calculate_features(test_df)
x_test = test_df.drop(droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_11 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
test_df['transactiondate'] = '2016-12-31'
calculate_features(test_df)
x_test = test_df.drop(droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_12 = model.predict(dtest, ntree_limit=model.best_ntree_limit)



output = pd.DataFrame({'ParcelId': sample_submission['ParcelId'].astype(np.int32),
           '201610': test_10, '201611': test_11, '201612': test_12,
           '201710': test_10, '201711': test_11, '201712': test_12})

output.to_csv('../data/feature_64201.csv', index=False)